# Eye of the Emergency

## Libraries and dataset import

In [1]:
import pandas as pd

# ML


In [2]:
train_df = pd.read_csv('train_tweets.csv')
test_df = pd.read_csv('test_tweets.csv')

## Data Preprocess

In [3]:
unique_count = pd.DataFrame(pd.value_counts(train_df.text))
unique_count = unique_count.reset_index()
unique_count.columns = ['text', 'count']

In [4]:
#unique_count.head()

In [5]:
train_df_unique = pd.DataFrame(train_df.text.unique(), columns = ['text'])
#train_df_unique.head()

In [6]:
train_df_unique = train_df_unique.merge(unique_count)
#train_df_unique.head()

In [7]:
train_df_count = train_df.merge(unique_count)

In [8]:
#train_df_count.head()

In [9]:
lg = pd.DataFrame([len(txt) for txt in train_df.text], columns = ['len_text'])
#lg.head()

In [10]:
train_df_len = pd.concat([train_df_count, lg], axis = 1)
#train_df_len.head()

In [11]:
df = train_df_len.copy()

### Location Encoder

In [12]:
encode_location = pd.DataFrame([int(type(ct)==str) for ct in df.location], columns=['location_enc'])

In [13]:
df = pd.concat([df, encode_location], axis=1)
#df.head()

## Data Visualization

In [14]:
stopwords = pd.read_csv('stopwords.txt', header = None)
stopwords.columns=['words']

In [15]:
stopwords = [wrd for wrd in stopwords.words]

In [16]:
stopwords.append('http')
stopwords.append('https')
stopwords.append('Û_')
stopwords.append('amp')

In [17]:
text = [txt for txt in df.text]
#text

In [18]:
my_lst_str = ' '.join(map(str, text))
#my_lst_str

In [19]:
text_0 = [txt for txt in df.text[df.target == 0]]
text_1 = [txt for txt in df.text[df.target == 1]]

In [20]:
my_lst_str_0 = ' '.join(map(str, text_0))
my_lst_str_1 = ' '.join(map(str, text_1))

In [21]:
#my_lst_str_0
#my_lst_str_1

## ML : SVM avec sklearn

In [22]:
# ML
from sklearn.datasets import load_iris

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [23]:
iris  = load_iris()
#iris.data
#iris.target_names
#iris.feature_names

In [24]:
iris_df = pd.DataFrame(iris.data, columns = iris.feature_names)

In [25]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [26]:
a = iris_df.loc[:,'petal length (cm)']
b = iris_df.loc[:,'petal width (cm)']

In [27]:
x = list(zip(a, b))
y = iris.target

In [28]:
#fractionner dataset (train-test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)

In [29]:
#instanciation
model_SVC = SVC(kernel = 'linear', gamma = 'scale', shrinking = False)

In [30]:
#training
model_SVC.fit(x_train, y_train)

SVC(kernel='linear', shrinking=False)

In [31]:
#calcul de la précision
model_SVC.score(x_test, y_test)

0.9666666666666667

In [32]:
#Prédiction
longueur = 2.5
largeur = 0.75
prediction = model_SVC.predict([[longueur, largeur]])

In [33]:
#affichage des résultats
resultat = "Résultat : "
if prediction[0] == 0:
    resultat = resultat + "setosa"
if prediction[0] == 1:
    resultat = resultat + "versicolor"
if prediction[0] == 2:
    resultat = resultat + "virginica"

In [34]:
resultat

'Résultat : versicolor'